In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'C:\\spark-3.3.1-bin-hadoop2'

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, cast, count, isnan, isnull, when


In [39]:
spark = SparkSession.builder.appName("diabet").master("local[2]").getOrCreate()

In [40]:
spark

In [ ]:
spark.sparkContext.setSystemProperty()

In [55]:
rawd = spark.read.csv("../drang/Downloads/diabetes.csv", header=True)
df = spark.range(0,20)
print(df.rdd.getNumPartitions())

2


In [56]:
rawd.rdd.getNumPartitions()

1

In [54]:
rawd.write.bucketBy(3,"age")
rawd.r

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [10]:
columnname = rawd.columns

In [11]:
rawd.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [14]:
rawf = rawd.select(*(col(c).cast('float').alias(c) for c in rawd.columns))

In [15]:
rawf.printSchema()

root
 |-- Pregnancies: float (nullable = true)
 |-- Glucose: float (nullable = true)
 |-- BloodPressure: float (nullable = true)
 |-- SkinThickness: float (nullable = true)
 |-- Insulin: float (nullable = true)
 |-- BMI: float (nullable = true)
 |-- DiabetesPedigreeFunction: float (nullable = true)
 |-- Age: float (nullable = true)
 |-- Outcome: float (nullable = true)



In [16]:
rawf.select([count(when(col(c).isNull(),c)).alias(c) for c in rawf.columns]).show()

+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+
|          0|      0|            0|            0|      0|  0|                       0|  0|      0|
+-----------+-------+-------------+-------------+-------+---+------------------------+---+-------+



In [17]:
#if theres null use imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCol='Age', outputCol='newage')
model = imputer.fit(rawf)
data = model.transform(rawf)

In [22]:
data.rdd.getNumPartitions()
d = data.repartition(5)
d.rdd.getNumPartitions()
d

5

In [17]:

colsi = rawf.drop(data.Outcome)

colsi = colsi.columns
colsi

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age']

In [18]:
from pyspark.ml.feature import VectorAssembler
n= VectorAssembler(inputCols=colsi, outputCol="feature")
newd = n.transform(rawf)
newd.show(truncate=False)

+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+-----------------------------------------------------------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin|BMI |DiabetesPedigreeFunction|Age |Outcome|feature                                                                |
+-----------+-------+-------------+-------------+-------+----+------------------------+----+-------+-----------------------------------------------------------------------+
|6.0        |148.0  |72.0         |35.0         |0.0    |33.6|0.627                   |50.0|1.0    |[6.0,148.0,72.0,35.0,0.0,33.599998474121094,0.6269999742507935,50.0]   |
|1.0        |85.0   |66.0         |29.0         |0.0    |26.6|0.351                   |31.0|0.0    |[1.0,85.0,66.0,29.0,0.0,26.600000381469727,0.35100001096725464,31.0]   |
|8.0        |183.0  |64.0         |0.0          |0.0    |23.3|0.672                   |32.0|1.0    |[8.0,183.0,64.0,0.0,0.0,23.29999923

In [19]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='feature', outputCol="scaledfeat")
model = scaler.fit(newd)
scaledd = model.transform(newd)


In [20]:
newdata = scaledd.select('scaledfeat','Outcome')

In [21]:
newdata.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|scaledfeat                                                                                                                                               |Outcome|
+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|[1.7806383732194306,4.628960915766174,3.7198138711154307,2.1940523222807116,0.0,4.261709202425419,1.8923810993699686,4.251616970894646]                  |1.0    |
|[0.29677306220323846,2.658524850271114,3.4098293818558116,1.8179290670325896,0.0,3.373853320188119,1.0593713140527197,2.6360025219546803]                |0.0    |
|[2.3741844976259077,5.723647618818986,3.306501218769272,0.0,0.0,2.955292430788826,2.028197980632078,2.721034861372573]                                   |1.0    |
|[0.296773062203

In [97]:
from pyspark.ml.feature import ChiSqSelector
chi = ChiSqSelector(numTopFeatures=6, outputCol="selectedcols", featuresCol="scaledfeat", labelCol="Outcome")
Sdata = chi.fit(newdata).transform(newdata)

In [102]:
Sdata.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------------------------------------------------------------------------------------------------------------+
|scaledfeat                                                                                                                                               |Outcome|selectedcols                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------------------------------------+-------+-------------------------------------------------------------------------------------------------------------------+
|[1.7806383732194306,4.628960915766174,3.7198138711154307,2.1940523222807116,0.0,4.261709202425419,1.8923810993699686,4.251616970894646]                  |1.0  

In [103]:
columnname #3rd and 7th- BP & DIABETESPEDIGREE REMOVED BY CHISQ FEAT SELECTOR

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [173]:
Ssdata= Sdata.drop("scaledfeat")
Ssdata.show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------+
|Outcome|selectedcols                                                                                                       |
+-------+-------------------------------------------------------------------------------------------------------------------+
|1.0    |[1.7806383732194306,4.628960915766174,2.1940523222807116,0.0,4.261709202425419,4.251616970894646]                  |
|0.0    |[0.29677306220323846,2.658524850271114,1.8179290670325896,0.0,3.373853320188119,2.6360025219546803]                |
|1.0    |[2.3741844976259077,5.723647618818986,0.0,0.0,2.955292430788826,2.721034861372573]                                 |
|0.0    |[0.29677306220323846,2.783631902048578,1.4418058117844677,0.8156606685129459,3.564108203936454,1.785679127775751]  |
|1.0    |[0.0,4.284916523378148,2.1940523222807116,1.4577765139380312,5.466656799498205,2.806067200790466]            

In [23]:
#traib test split
train, test = newdata.randomSplit([0.7,0.3])
test.count()
newdata.show()

+--------------------+-------+
|          scaledfeat|Outcome|
+--------------------+-------+
|[1.78063837321943...|    1.0|
|[0.29677306220323...|    0.0|
|[2.37418449762590...|    1.0|
|[0.29677306220323...|    0.0|
|[0.0,4.2849165233...|    1.0|
|[1.48386531101619...|    0.0|
|[0.89031918660971...|    1.0|
|[2.96773062203238...|    0.0|
|[0.59354612440647...|    1.0|
|[2.37418449762590...|    1.0|
|[1.18709224881295...|    0.0|
|[2.96773062203238...|    1.0|
|[2.96773062203238...|    0.0|
|[0.29677306220323...|    1.0|
|[1.48386531101619...|    1.0|
|[2.07741143542266...|    1.0|
|[0.0,3.6906580274...|    1.0|
|[2.07741143542266...|    1.0|
|[0.29677306220323...|    0.0|
|[0.29677306220323...|    1.0|
+--------------------+-------+
only showing top 20 rows



In [155]:
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier

In [157]:
logreg = LogisticRegression(labelCol="Outcome", featuresCol="scaledfeat", maxIter=500, regParam=0.2, elasticNetParam=0.8)
model = logreg.fit(train)
preds = model.transform(test)
evals = model.evaluate(test)

In [166]:
preds.show()

print(evals.accuracy, evals.areaUnderROC)


+--------------------+-------+--------------------+--------------------+----------+
|          scaledfeat|Outcome|       rawPrediction|         probability|prediction|
+--------------------+-------+--------------------+--------------------+----------+
|(8,[0,1,6,7],[0.5...|    0.0|[0.89631638308175...|[0.71019193039587...|       0.0|
|(8,[0,1,6,7],[0.8...|    0.0|[0.91518848394469...|[0.71406071260696...|       0.0|
|(8,[1,5,6,7],[2.2...|    0.0|[0.94821466045483...|[0.72075599011862...|       0.0|
|(8,[1,5,6,7],[4.0...|    1.0|[0.67456919794224...|[0.66252552284129...|       0.0|
|(8,[1,5,6,7],[4.4...|    1.0|[0.62738894578490...|[0.65189717809053...|       0.0|
|(8,[1,5,6,7],[4.5...|    1.0|[0.60851684492196...|[0.64760240016663...|       0.0|
|(8,[1,6,7],[2.940...|    0.0|[0.84913613092441...|[0.70038589482832...|       0.0|
|[0.0,2.8461854279...|    0.0|[0.86329020657161...|[0.70334761411082...|       0.0|
|[0.0,2.8461854279...|    0.0|[0.86329020657161...|[0.70334761411082...|    

In [167]:
"""from pyspark.mllib.evaluation import BinaryClassificationMetrics
#convert to rdd cox of mllib pkg
predsa = preds.select('Outcome','prediction').rdd.map(lambda row: (float(row['probability'][1]), float(row['Outcome'])))
metrics = BinaryClassificationMetrics(predsa)"""

"from pyspark.mllib.evaluation import BinaryClassificationMetrics\n#convert to rdd cox of mllib pkg\npredsa = preds.select('Outcome','prediction').rdd.map(lambda row: (float(row['probability'][1]), float(row['Outcome'])))\nmetrics = BinaryClassificationMetrics(predsa)"

In [168]:
"""print("AUC", metrics.areaUnderPR)"""

'print("AUC", metrics.areaUnderPR)'

In [169]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [170]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
metric = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome", metricName="areaUnderROC")
lrroc=metric.evaluate(predy)
lrroc

0.8354993424615148

In [171]:
metrics = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="Outcome", metricName="accuracy")
lracc = metrics.evaluate(predy)
lracc

0.5991379310344828

In [172]:

rf = RandomForestClassifier(labelCol="Outcome", featuresCol="scaledfeat", numTrees=20)
rfmod = rf.fit(train)
pred = rfmod.transform(test)

In [173]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [174]:
rfroc = metric.evaluate(predy)

In [175]:
rfacc = metrics.evaluate(predy)

In [176]:
ml = GBTClassifier(labelCol="Outcome", featuresCol=("scaledfeat"), maxDepth=10, maxIter=100)
model = ml.fit(train)


In [177]:
preds = model.transform(test)

#convert SparseVector -> DenseVector  which error occured during chisq selector

In [178]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

gbtroc=metric.evaluate(predy)

In [179]:
gbtacc = metrics.evaluate(predy)

In [180]:
roc = [lrroc, rfroc, gbtroc]
acc = [lracc, rfacc, gbtacc]
print(f"lrroc:{lrroc},  rfroc:{rfroc}, gbtroc:{gbtroc}")
print(f"lracc:{lracc},  rfacc:{rfacc}, gbtacc:{gbtacc}")

lrroc:0.8354993424615148,  rfroc:0.8354993424615148, gbtroc:0.7328459812794925
lracc:0.5991379310344828,  rfacc:0.5991379310344828, gbtacc:0.7068965517241379


In [205]:
from pyspark.mllib.stat import Statistics
n = newdata.drop("Outcome")
n = newdata.select(newdata.scaledfeat).rdd.map(lambda row : row[0:])
summary = Statistics.colStats(n)

summary.mean() #variance, norm

array([1.14110788, 3.7811896 , 3.57027057, 1.28737326, 0.69243932,
       4.05782949, 1.42419431, 2.82655025])

In [206]:
correlat = Statistics.corr(n, method='pearson')
import pandas as pd
pddf= pd.DataFrame(correlat, columns=columnname[:-1], index=columnname[:-1])
pddf


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Pregnancies,1.000000,0.129459,0.141282,-0.081672,-0.073535,0.017683,-0.033523,0.544341
Glucose,0.129459,1.000000,0.152590,0.057328,0.331357,0.221071,0.137337,0.263514
BloodPressure,0.141282,0.152590,1.000000,0.207371,0.088933,0.281805,0.041265,0.239528
SkinThickness,-0.081672,0.057328,0.207371,1.000000,0.436783,0.392573,0.183928,-0.113970
Insulin,-0.073535,0.331357,0.088933,0.436783,1.000000,0.197859,0.185071,-0.042163
BMI,0.017683,0.221071,0.281805,0.392573,0.197859,1.000000,0.140647,0.036242
DiabetesPedigreeFunction,-0.033523,0.137337,0.041265,0.183928,0.185071,0.140647,1.000000,0.033561
Age,0.544341,0.263514,0.239528,-0.113970,-0.042163,0.036242,0.033561,1.000000


In [218]:
from pyspark.ml.stat import Correlation
n = newd.drop('Outcome')
n = n.drop('features')
summary = Correlation()
print(n)
columnname=columnname[:-1]
a=summary.corr(dataset=n.feature, column=columnname) #variance, norm
a

DataFrame[Pregnancies: float, Glucose: float, BloodPressure: float, SkinThickness: float, Insulin: float, BMI: float, DiabetesPedigreeFunction: float, Age: float, feature: vector]


Traceback (most recent call last):
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "C:\spark-3.3.1-bin-hadoop2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
TypeError: cannot pickle '_thread.RLock' object


PicklingError: Could not serialize object: TypeError: cannot pickle '_thread.RLock' object

In [217]:
n.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+----+--------------------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction| Age|             feature|
+-----------+-------+-------------+-------------+-------+----+------------------------+----+--------------------+
|        6.0|  148.0|         72.0|         35.0|    0.0|33.6|                   0.627|50.0|[6.0,148.0,72.0,3...|
|        1.0|   85.0|         66.0|         29.0|    0.0|26.6|                   0.351|31.0|[1.0,85.0,66.0,29...|
|        8.0|  183.0|         64.0|          0.0|    0.0|23.3|                   0.672|32.0|[8.0,183.0,64.0,0...|
|        1.0|   89.0|         66.0|         23.0|   94.0|28.1|                   0.167|21.0|[1.0,89.0,66.0,23...|
|        0.0|  137.0|         40.0|         35.0|  168.0|43.1|                   2.288|33.0|[0.0,137.0,40.0,3...|
|        5.0|  116.0|         74.0|          0.0|    0.0|25.6|                   0.201|3